Data Wrangling Methodology:

Billboard Data

I first need to make sure the datasets collected from Billboard are clean. The data for each year is kept in CSV files in the Datasets Folder from the root.

There should be 20 csvs with 100 records each. 

Methodology:

Concatenate all csvs into one large df. I can then check for missing values and duplicates. 


In [26]:
#Import packages
import pandas as pd
import glob

In [27]:
path = 'Billboard Data'
all_files = glob.glob(path + "*/.csv")

In [28]:
l1 = []

In [29]:
bill_datasets = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20']

In [30]:
#concatenate all csvs
for year in bill_datasets:
    
    df = pd.read_csv(f'Billboard Data/{year}.csv', index_col=None, header = 0)
    l1.append(df)

billboard_df = pd.concat(l1, axis = 0, ignore_index = True)

In [31]:
billboard_df.head(102)

,Artist,Title
0,Lifehouse,Hanging By A Moment
1,Alicia Keys,Fallin’
2,Janet Jackson,All For You
3,Train,Drops Of Jupiter (Tell Me)
4,Jennifer Lopez feat. Ja Rule,I’m Real
...,...,...
97,QB’s Finest feat. Nas and Braveheart’s,Oochie Wally
98,Enrique Iglesias,Hero
99,Fuel,Hemorrhage (In My Hands)
100,Nickelback,How Your Remind Me


In [32]:
billboard_df.describe()

,Artist,Title
count,1999,1998
unique,1215,1766
top,Taylor Swift,Beautiful
freq,21,4


In [52]:
#Check for duplicates
duplicates = billboard_df[billboard_df.duplicated(subset=['Artist','Title'], keep=False)]

duplicates.sort_values(by=['Artist', 'Title'])



0       False
1       False
2       False
3       False
4       False
        ...  
1994    False
1995    False
1996    False
1997    False
1998    False
Length: 1875, dtype: bool

In [36]:
billboard_df.drop_duplicates(subset = ['Artist','Title'], inplace=True)

In [37]:
billboard_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1875 entries, 0 to 1998
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Artist  1875 non-null   object
 1   Title   1874 non-null   object
dtypes: object(2)
memory usage: 43.9+ KB


We found 124 duplicates and removed them.

In [49]:
dups = billboard_df[billboard_df.duplicated(subset=['Artist','Title'], keep=False)]
dups

,Artist,Title


In [38]:
billboard_df.describe()



,Artist,Title
count,1875,1874
unique,1215,1766
top,Rihanna,Beautiful
freq,17,4


Time to deal with missing values. There was 1 initial missing Artist name (we would have expected 2000 values in the Artist column and found 1999). Serendipitously, upon inspection of the first dataset I found it contained 99 values for Artist and Title. There are therefor only 99 Artist : Title pairs in the 2001 dataset.

However, this still leaves the one Artist that is missing a Title. Let's see what that is..

In [39]:
billboard_df.isna().sum()

Artist    0
Title     1
dtype: int64

In [40]:
missing = billboard_df[billboard_df['Title'].isna()]

In [41]:
missing

,Artist,Title
1239,"Mike WiLL Made-It FeaturingMiley Cyrus, Wiz Kh...",NaN


The missing value is actually pretty obvious. It is the song '23' by Mike Will Made-it feat. Miley Cyrus, Wiz Khalifa, etc. 

Lucky the Artist column was so specific! Lets add the value back in to complete the df. 

In [42]:
billboard_df.loc[1239,'Title'] = '23'

In [43]:
billboard_df.describe()

,Artist,Title
count,1875,1875
unique,1215,1766
top,Rihanna,Beautiful
freq,17,4


In [44]:
#Found an incorrect Artist name. Need to correct All for you by Janet to all for you by Janet Jackson

billboard_df.loc[2,'Artist'] = 'Janet Jackson'

In [45]:
#replace all instance of Pink, the artist) with the correct spelling: "p!ink"

billboard_df.replace(to_replace = 'Pink', value = 'P!nk',inplace = True)

In [46]:
billboard_df.head()

,Artist,Title
0,Lifehouse,Hanging By A Moment
1,Alicia Keys,Fallin’
2,Janet Jackson,All For You
3,Train,Drops Of Jupiter (Tell Me)
4,Jennifer Lopez feat. Ja Rule,I’m Real


In [47]:
#Save billboard_df to csv. 

billboard_df.to_csv('BillboardFullDataset.csv')